# Stock Price Prediction using LSTM
# 
## This notebook demonstrates how to predict stock prices using a Long Short-Term Memory (LSTM) neural network.

## This week's (Week 3) tasks:
### Write a function to load and process a dataset with multiple features with the following requirements:
1. Write a function to display stock market financial data using candlestick chart. You can use the following tutorial: https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh but again, you will need to explain in details all the code in your program (including the meanings of the arguments in a function call). Furthermore, you need to include an option in your function to allow each candle stick to express the data of n trading days (n ≥ 1).
2. Write a function to display stock market financial data using boxplot chart. This is particularly useful when you are trying to display your data for a moving window of n consecutive trading days. Again, feel free to use online resources that teach you how to do this but you need to add comments to explain your codes and the parameters you use.
3. Upload your Task 3 Report (as a PDF file) to the project Wiki before the deadline and email your project leader to notify that it is ready for viewing and feedback.

### Create a Virtual Environment using `Conda` (If you have your own `Virutal Environment` your then ignore this !)

In [ ]:
# To activate the conda environment, copy the below content to `conda-config.sh` and run the following command in the terminal:
# sh conda-config.sh

'''conda-config.sh

conda create -n cos30019_env_w3_v0.2 python=3.10.9
conda activate cos30019_env_w3_v0.2
#Check current environment
conda info --envs
#Check current python version
python --version

'''

### Install essential libraries.

In [ ]:
!pip install -r requirements.txt
# !pip install numpy
# !pip install matplotlib
# !pip install pandas
# !pip install tensorflow
# !pip install scikit-learn
# !pip install pandas-datareader
# !pip install yfinance
# !pip install -q talib
# !pip install joblib

### Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import pandas_datareader as pdr
# import datetime as dt
import yfinance as yf
import talib as ta
import mplfinance as mpf
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from sklearn import metrics

# import tensorflow as tf
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import Dense, LSTM, Dropout, InputLayer, Input, Activation
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.utils import plot_model

import pickle
import os
from typing_extensions import Annotated, Doc, TypeVar, Literal, Tuple

### Declare essential variables
- Define the start date, the end date and the ticker (stock) that we wanted to download from `yfinance`.
- Define the split ratio for splitting the train and test data, and the number of days for looking back.
- Define the directory for storing the raw and prepared datasets.
- Define the files that are used for storing essential information, such as raw data imported from yfinance and the processed data.

In [ ]:
# Start, end and ticker for the datasets
start="2015-01-01"
end="2023-08-25"
ticker="TSLA"

# Split ratio, 0.8 equals to 80% data for training and 20% for testing
split_ratio = 0.8

# Number of days to look back for the prediction, could be changed to any value
num_look_back_days = 30

# Define entry directory for the raw and prepared datasets
ENTRY_POINT = f"datasets/{ticker}/from_{start}_to_{end}"
RAW_DATA_DIRECTORY = os.path.join(ENTRY_POINT, "data")
PREPARED_DATA_DIRECTORY = os.path.join(ENTRY_POINT, "prepared-data")

# Raw data to be saved as the same for
RAW_CSV_FILE = os.path.join(RAW_DATA_DIRECTORY, f"raw_data_from_{start}_to_{end}_of_{ticker}_stock.csv")

# Prepared data to be saved as the same for
PREPARED_CSV = os.path.join(PREPARED_DATA_DIRECTORY, f"prepared_data_from_{start}_to_{end}_of_{ticker}_stock.csv")
PREPARED_TRAIN_ALL = os.path.join(PREPARED_DATA_DIRECTORY, f"xytrain_data_{start}-{end}-{ticker}_stock.npz")
PREPARED_TRAIN_DATASET = os.path.join(PREPARED_DATA_DIRECTORY, f"train_dataset_of_{ticker}_from_{start}_to_{end}.csv")
PREPARED_TEST_DATASET = os.path.join(PREPARED_DATA_DIRECTORY, f"test_dataset_of_{ticker}_from_{start}_to_{end}.csv")
PREPARED_SCALER_FEATURE = os.path.join(PREPARED_DATA_DIRECTORY, f"feature_scaler_of_{ticker}_from_{start}_to_{end}.pkl")
PREPARED_SCALER_TARGET = os.path.join(PREPARED_DATA_DIRECTORY, f"targe_scaler_of_{ticker}_from_{start}_to_{end}.pkl")
PREPARED_TRAIN_ARRAY = os.path.join(PREPARED_DATA_DIRECTORY, f"xytrain_train_array_of_{ticker}_from_{start}_to_{end}.npz")
PREPARED_TEST_ARRAY = os.path.join(PREPARED_DATA_DIRECTORY, f"xytrain_test_array_of_{ticker}_from_{start}_to_{end}.npz")

### Ensure that directory must exists
- check_directory_exists() & create_directory(): The purpose of these two functions are two check if the given dir_path is exists or not using os.path.isdir(), if not then the check_directory_exists() will return a False and use create_directory() to create a folder based on the given dir_path, otherwise return True. 
- check_file_exists(): This function will check if the file exists using os.path.exists() function, it will return True if exists otherwise return False.

In [ ]:
def check_directory_exists(dir_path: Annotated[str, Doc("The path to the directory to be checked")]) -> bool:
    return True if os.path.isdir(dir_path) else False

def create_directory(dir_path: Annotated[str, Doc("The path to the directory to be created")]) -> None:
    os.makedirs(dir_path)

def check_file_exists(file_path: Annotated[str, Doc("The path to the file to be checked")]) -> bool:
    return True if os.path.exists(file_path) else False

### Utility function to load and save data objects
**The load_and_save_object() function does the following procedure:**
- If the mode is set to save, the function will start opening up the file defined in fn parameter. Once the file is successfully opened, it will write all of the given objects defined in obj by using pickle.dump() to ensure that the object will be type-safe when loading the object from the file.
- If the mode is set to load, the function will start reading from the file defined in fn and load the object using pickle.load() function

In [ ]:
T = TypeVar("T", bound=Literal["load", "save"])
U = TypeVar("U", any, object)
def save_or_load_object(obj: Annotated[U | None, Doc("The object to be saved or loaded")],
                         fn: Annotated[str, Doc("The filename to the directory to be checked")],
                         mode: Annotated[T, Doc("The mode to be used")]) -> U | None:
    # Save the object to the file if the mode is save
    if mode == "save":
        with open(fn, "wb") as i:
            pickle.dump(obj, i)
    # Load the object from the file if the mode is load
    elif mode == "load":
        # Check if the object is None, if so then load the object from the file
        if obj is None:
            with open(fn, "rb") as i:
                return pickle.load(i)
        else:
            raise ValueError("obj must be None when mode is load")
    # Raise an error if the mode is invalid
    else:
        raise ValueError("Invalid mode")

### Load data function. Check if exists data, if not then load and save from the `yfinance`
- **The major purpose of this function is to check if the unprocessed CSV file that contains the stock data from the start date and end date is existed in the file or not, if it is not existed then it will download the data from yfinance and save it to the directory that contains unprocessed CSV data. Otherwise, it will load the unprocessed CSV File. The following procedures are:**
    - The code will check if the directory that contains unprocessed CSV file exists, if not then it will start to create the directory and download the CSV from the yfinance. Once it’s downloaded then it will convert the data to CSV Format and save it to the created directory.
    - However, in the case where the directory exists then it will read the file in that directory.


In [ ]:
def load_data(start: Annotated[str, Doc("The start date of the stock to be loaded")],
              end: Annotated[str, Doc("The end date of the stock to be loaded")],
              ticker: Annotated[str, Doc("The ticker symbol of the stock to be loaded")] = "CBA.AX"):
    # Check if the raw data directory exists, if not then create it
    if not check_directory_exists(RAW_DATA_DIRECTORY):
        create_directory(RAW_DATA_DIRECTORY)
        data = yf.download(ticker, start=start, end=end)
        data.to_csv(RAW_CSV_FILE)
    else:
        # Load the data from the local machine
        data = pd.read_csv(RAW_CSV_FILE)
    return data

### Split data by ratio
**This function will do the following procedures:**
- If is_split_by_date is set to True: This function will perform calculation based on the split ratio (0.8 will equal 80% of train data and 20% of test data). Once the calculation is finished, the function will allocate the train and test data to the train_data and test_data variables.
- If is_split_by_date is set to False: This function will use the built-in function provided by scikit-learn, which is train_test_split() to split the data randomly based on the given ratio value. The random_state is fixed at 42 to make sure that the splitted data is reproducible.
- Once the data has been successfully splitted, it will then print out the shape (x,y) of both train and test data and return it to the user for further usages.

In [ ]:
def split_data_by_ratio(data: Annotated[pd.DataFrame, Doc("The data to be split")],
                        ratio: Annotated[float, Doc("The ratio to be used in percentage: 0.8 for 80% train and 20% test")],
                        is_split_by_date: Annotated[bool, Doc("Choose to split by date or random")] = True) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Check if the data is split by date
    if is_split_by_date:
        # Calculate the train data size of the train data
        train_size = int(len(data) * ratio)
        # Split the data into train and test data
        train_data, test_data = data.iloc[:train_size], data.iloc[train_size:]
    else:
        # Split the data into train and test data randomly
        train_data, test_data = train_test_split(data,train_size=1 - (1 - ratio), test_size= 1 - ratio, random_state=42)

    # Print the shape of the train and test data
    print(f"Train data shape: {train_data.shape}")
    print(f"Test data shape: {test_data.shape}")

    return train_data, test_data

### Scaling data using `MinMaxScaler()`
- **This function will check if the is_scale equals to True or not, if it is not then the function will simply return back the data without any scaling data. Otherwise, it will do the following procedures:**
    - Scaling data: This function will use the MinMaxScaler class provided by scikit-learn to scale the data to the range of (0, 1) by setting the feature_range to a tuple of (0, 1).
    - Reshaping data: After the data is scaled, the data will be converted into a two-dimensional array using reshape(-1, 1) function. This procedure is essential as the fit_transform() function only accepts a 2D array.
    - Fitting and returning the scaled array: Once the array is transformed to 2D array, it will be fitted using the fit_transform() function provided by class MinMaxScaler. Once the transform is finished we will return the scaled data with the instance of MinMaxScaler for further usage.

In [ ]:
def scaling_data(data: Annotated[pd.DataFrame, Doc("The data to be scaled")], is_scale: Annotated[bool, Doc("Choose to scale or not, default set to True")] = True) -> Tuple[pd.DataFrame, MinMaxScaler | None]:
    if is_scale:
        # Using MinMaxScaler to scale the data to the range of (0, 1)
        scaler = MinMaxScaler(feature_range=(0, 1))
        if len(data.shape) == 1:
            # If the data array is 1D, convert to 2D array
            data = data.values.reshape(-1, 1)
        # Fit the 2D-transformed data into the scaler
        scaled_data = scaler.fit_transform(data)
        return scaled_data, scaler
    else:
        return data, None

### Validate data
**This function will check if the directory that contains processed data exists, if not then it will create a directory used for storing processed data.
Once the directory is created, it will check the existence of the processed CSV file. If the CSV file exists, it will read the CSV file using pd.read_csv() and return it. Otherwise, it will start the following procedure**:
- Read the unprocessed CSV file: It will read the unprocessed CSV file stored in the unprocessed folder.
- Convert and set index: After the unprocessed CSV file is successfully read, it will convert the value of Date column into the DateTime type and set the value inside the Date column as index. This will make it easier to operate time-based searching, simplify the plotting procedure and enhance the data organization.
- Adding essential indicators to the data: This function will also add several required technical analysis indicators to the data, such as Relative Strength Index (RSI) and several different date ranges (20 days, 100 days and 150 days) of Exponential Moving Averages (EMA) using the built-in functions provided by TA-lib.
- Calculate target price: After adding indicators to the data, it will start calculating the target price by subtracting the value of Adjustment Close Price and the Open Price. Once subtracted, it will shift the value back by one to assume that is the targeted price.
- Indicate if the value is increased or not: It will calculate the TargetClass based on the target price value and indicate if the price is increased or not, if the price is increased then the TargetClass value will be 1, otherwise it will be 0.
- Drop all undefined values: By using dropna() function, it will look up all of the undefined (NaN) values and drop it.
- Save processed data: Once the datasets are processed, it will save those datasets into a CSV File by using to_csv() function and return the processed datasets to the user.

In [ ]:
def validate_data() -> pd.DataFrame:
    # Check if the prepared data directory exists, if not then create it
    if not check_directory_exists(PREPARED_DATA_DIRECTORY):
        create_directory(PREPARED_DATA_DIRECTORY)

    if check_file_exists(PREPARED_CSV):
        print('Loading Prepared Data...')
        df = pd.read_csv(PREPARED_CSV)
    else:
        print('Processing Raw Data...')
        # Read the raw data
        df = pd.read_csv(RAW_CSV_FILE)
        # Convert the date column to datetime
        if not isinstance(df.index, pd.DatetimeIndex):
            df['Date'] = pd.to_datetime(df['Date'])
            # Set the date column as the index
            df.set_index('Date', inplace=True)
        print("Type of index after converted: ", type(df.index))
        # Adding RSI, EMA20(EMA for 20 days), EMA100(EMA for 100 days), EMA200(EMA for 200 days)
        df['Close_RSI'] = ta.RSI(df['Close'], timeperiod=14)
        df['Close_EMA20'] = ta.EMA(df['Close'], timeperiod=20)
        df['Close_EMA100'] = ta.EMA(df['Close'], timeperiod=100)
        df['Close_EMA200'] = ta.EMA(df['Close'], timeperiod=200)

        # Calculate the target value by subtracting the open price
        # and the adjusted close price and shifting it by 1 day
        df['Target'] = df['Adj Close'] - df['Open']
        df['Target'] = df['Target'].shift(-1)

        # Convert the target class to binary class if the target is greater than 0
        df['TargetClass'] = np.where(df['Target'] > 0, 1, 0)
        # Shift the adjusted close price by 1 day
        df['TargetNextClose'] = df['Adj Close'].shift(-1)

        # Drop the NaN values
        df.dropna(inplace=True)

        # Convert to csv and save to the folder
        df.to_csv(PREPARED_CSV, index=False)
    return df


### Create datasets
**The given procedure, which is the heart to execute successfully the program, does the following steps:**
- Load and validate data: The procedure initially calls to the defined load_data() function, which will either load the existing raw data or download the data if it does not exist. Once the data is loaded successfully, it will call the validate_data() function to process and validate the data to be in the right format, which adds in extra technical analysis indicators and target values.
- Check the existence of the prepared datasets: After the data is validated, it will check if the train and test datasets exist. If the datasets are existed, it will do the following steps:
    - Load from the processed CSV file the existing datasets, including train and test datasets.
    - Print out the shapes of both train and test datasets for validation.
    - Load the saved scaler values of both feature and target.
    - Retrieve the value from the x and y array of both train and test arrays.
- In case where the datasets are not existed, the following logic will be executed:
    - The validated data will be split into train and test data using the defined split_data_by_ratio() function. With the split rate defined at 0.8 (80% of train data, 20% of test data).
    - Scale the training data for the features using the defined scaling_data() function and based on the defined feature_cols columns.
    - Make training arrays by creating a sequence for each sample based on the defined variable num_look_back_days.
    - Scale the testing data for the features and target using the defined scaling_data() function and based on the defined feature_cols and target_cols columns.
    - Make test arrays by creating a sequence of each sample based on the defined variable num_look_back_days.
    - Save the train and test data to a CSV file by using the to_csv()function from pandas and save the feature and target scalers using the defined save_or_load_object() function.
    - Save the created train and test arrays to a .npz file using savez()function from numpy.


In [ ]:
data = load_data(start, end, ticker)
df = validate_data()

if check_file_exists(PREPARED_TRAIN_DATASET) and check_file_exists(PREPARED_TEST_DATASET):
    print('Loading Existed Train and Test Data...')
    train_data = pd.read_csv(PREPARED_TRAIN_DATASET)
    test_data = pd.read_csv(PREPARED_TEST_DATASET)

    print(f"Train data shape: {train_data.shape}")
    print(f"Test data shape: {test_data.shape}")

    train_feature_scale = save_or_load_object(None, PREPARED_SCALER_FEATURE, "load")
    train_target_scale = save_or_load_object(None, PREPARED_SCALER_TARGET, "load")
    train_arrays = np.load(PREPARED_TRAIN_ARRAY)
    x_train = train_arrays['x_train']
    y_train = train_arrays['y_train']

    test_arrays = np.load(PREPARED_TEST_ARRAY)
    x_test = test_arrays['x_test']
    y_test = test_arrays['y_test']
else:
    print('Processing Train and Test Data...')
    train_data, test_data = split_data_by_ratio(df, split_ratio)

    feature_cols = ['Open', 'High', 'Low', 'Close_RSI', 'Close_EMA20', 'Close_EMA100', 'Close_EMA200']
    target_cols = 'TargetNextClose'

    scaled_train_data, train_feature_scale = scaling_data(train_data[feature_cols])
    converted_2d_train_data = train_data[target_cols].values.reshape(-1, 1)
    scaled_train_target, train_target_scale = scaling_data(converted_2d_train_data)

    x_train, y_train = [], []
    for i in range(num_look_back_days, len(scaled_train_data)):
        x_train.append(scaled_train_data[i-num_look_back_days:i])
        y_train.append(scaled_train_target[i])

    x_train, y_train = np.array(x_train), np.array(y_train)

    scaled_test_data = train_feature_scale.transform(test_data[feature_cols])
    converted_2d_test_data = test_data[target_cols].values.reshape(-1, 1)
    scaled_test_target = train_target_scale.transform(converted_2d_test_data)

    x_test, y_test = [], []
    for i in range(num_look_back_days, len(scaled_test_data)):
        x_test.append(scaled_test_data[i-num_look_back_days:i])
        y_test.append(scaled_test_target[i])

    x_test, y_test = np.array(x_test), np.array(y_test)

    train_data.to_csv(PREPARED_TRAIN_DATASET, index=False)
    test_data.to_csv(PREPARED_TEST_DATASET, index=False)

    save_or_load_object(train_feature_scale, PREPARED_SCALER_FEATURE, "save")
    save_or_load_object(train_target_scale, PREPARED_SCALER_TARGET, "save")

    np.savez(PREPARED_TRAIN_ARRAY, x_train=x_train, y_train=y_train)
    np.savez(PREPARED_TEST_ARRAY, x_test=x_test, y_test=y_test)

### Type and Shape of the processed data

In [ ]:
print("Data Preparation Completed!")
print("Data: ", type(data))
print("Data: ", type(df))
print("Index: ", type(df.index))
print("Train Data: ", train_data.shape)
print("Test Data: ", test_data.shape)
print("Train Feature Scaler: ", type(train_feature_scale))
print("Train Target Scaler: ", type(train_target_scale))
print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)

### Raw data informations

In [ ]:
print(len(data))
data.head(3)
data.tail(3)

### Value counts for the transformed TargetClass

In [ ]:
print(len(df))
df.tail(3)
print(type(df.index))

### Length of the train data and the train data informations

In [ ]:
print(len(train_data))
train_data

### Length of the test data and the test data informations

In [ ]:
print(len(test_data))
test_data

### Expected and actual train / test radio

In [ ]:
print("Actual Ratio: ", split_ratio)
print("Received train ratio: ", len(train_data) / len(df))
print("Received test ratio: ", len(test_data) / len(df))

### The processed close price of the TSLA stock

In [ ]:

plt.figure(figsize=(12,6))
plt.plot(df.index, df['Close'])
plt.title(f'{ticker} Stock Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.show()

### TSLA Closing Price with 20 / 100 / 200 days index of EMA

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Close'], label='Close Price')
plt.plot(df.index, df['Close_EMA20'], label='20-day EMA', color="red")
plt.plot(df.index, df['Close_EMA100'], label='100-day EMA', color="orange")
plt.plot(df.index, df['Close_EMA200'], label='200-day EMA', color="green")
plt.title(f'{ticker} Stock Price with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

### RSI Index of the TSLA

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Close_RSI'])
plt.title(f'{ticker} RSI')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.axhline(y=70, color='r', linestyle='--')
plt.axhline(y=30, color='g', linestyle='--')
plt.grid(True)
plt.show()

### Draw Candlestick Chart

**In short, the draw_candlestick_chart() will take in the dataframe and create a copy, once the copy is created then it will resample the data by the n trading days if n is not None. It then adds in essential technical analysis strategies as additional values to display on the chart and draw the candlestick chart using mplfinance. The following steps display the procedure of the draw_candlestick_chart():**

- Dataframe copy: When the dataframe is passed in as an argument, this function will start copying the dataframe using the copy() function provided by pandas. This step is critically essential, as it will prevent any unwanted changes that could be executed to the real dataframe. 
- Data resample: The draw_candlestick_chart() will then check if the n parameter is not None, if the n parameter is not None then the function will start aggregating the data to find the maximum value of high price, minimum value of low price, first open price value, last close price value and the total of volumes within each resampling n period. Technical Analysis Strategies: Once the resampling procedure is finished, it will add three moving averages (MA) for the ‘Close’ price to the copied dataframe, which are 25 days, 100 days and 200 days moving average.
- Create Sub-Plot: The function also creates an array of make_addplot()instances, which will be used to display the moving averages (25 days, 100 days and 200 days) alongside the candlestick chart. However, before the chart is being appended to the array, it must be pre-processed by drop NaN values and reindexed for assuring safe data.
- Display the chart: Once the moving averages sub-plot procedure is finished, the function will utilize plot() function from mplfinance to display the candlestick chart. As shown in the codebase, this function will take in the dataframe that we have processed before, with some required parameters, which could be list as the type of the chart, the title of the chart, the style of the chart, the label name, the total volume of the data and the sub-plot that we have created before.

In [ ]:
def draw_candlestick_chart(df: Annotated[pd.DataFrame, Doc("The data to be plotted")],
                           n: Annotated[int | None, Doc("Resampling period in trading days for aggregation, default is None")] = None,
                           ):
    cp_df = df.copy()
    if n is not None:
        cp_df = cp_df.resample(f'{n}D').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        }).dropna()

    cp_df['MA25'] = cp_df['Close'].rolling(window=25).mean()
    cp_df['MA100'] = cp_df['Close'].rolling(window=100).mean()
    cp_df['MA200'] = cp_df['Close'].rolling(window=200).mean()


    sub_chart: mpf.make_addplot  = []
    if cp_df.MA25.dropna().shape[0] > 0:
        fixed_MA25 = cp_df.MA25.dropna().reindex(cp_df.index, fill_value=None)
        sub_chart.append(mpf.make_addplot(fixed_MA25, color='blue'))
    if cp_df.MA100.dropna().shape[0] > 0:
        fixed_MA100 = cp_df.MA100.dropna().reindex(cp_df.index, fill_value=None)
        sub_chart.append(mpf.make_addplot(fixed_MA100, color='orange'))
    if cp_df.MA200.dropna().shape[0] > 0:
        fixed_MA200 = cp_df.MA200.dropna().reindex(cp_df.index, fill_value=None)
        sub_chart.append(mpf.make_addplot(fixed_MA200, color='green'))

    mpf.plot(cp_df, type='candle', style='charles', ylabel='Price', ylabel_lower='Volume', volume=True, addplot=sub_chart, title=f'{ticker} Stock Price with Moving Averages')





In [ ]:
draw_candlestick_chart(data)

In [ ]:
draw_candlestick_chart(data, n=15)

In [ ]:
draw_candlestick_chart(data, n=60)

### Draw Box Chart

**The process of resampling data and the process of prepare data for the box plot is partially the same as the draw_candlestick_chart(), however, the key difference that separates the draw_box_chart() function is the way that it creates the sub-chart and the libraries that it uses for creating the chart. The below steps show the overall procedure of how the draw_box_chart() function work:**

- Dataframe copy: When the dataframe is passed in as an argument, this function will start copying the dataframe using the copy() function provided by pandas. This step is critically essential, as it will prevent any unwanted changes that could be executed to the real dataframe. 
- Data resample: The draw_candlestick_chart() will then check if the n parameter is not None, if the n parameter is not None then the function will start aggregating the data to find the maximum value of high price, minimum value of low price, first open price value, last close price value and the total of volumes within each resampling n period.
- Data preparation: After the data resample is finished, the function will start preparing data for displaying to the plot. First, the two arrays of chart_data and labels are created. The chart_data contains the close, open, low and high price for each data point and the labels will contain the corresponding date for each data point.
- Chart display: After preparing the data, the function will start creating and configuring the chart. First, the function will use the subplots() function provided by matplotlib to create a figure and axes. It will then initialize and create the box chart using the boxplot() function (which is also provided by matplotlib). The prepared chart_data array will be used as the main data to draw the box plot with different parameters are set to customize the appearance of the chart, which could be listed as: vertical alignment, patch artist, label , title, label for x and y axis, x-tick interval, the label of the x-tick and the rotation degree of the chart. The k parameter that is defined in the function will be used for determining the interval for displaying the labels at which days. Finally, the show() function of matplotlib is called to display the charts.

In [ ]:
def draw_box_chart(df: Annotated[pd.DataFrame, Doc("The data to be plotted")],
                    n: Annotated[int | None, Doc("Resampling period in trading days for aggregation, default is None")] = None,
                    k: Annotated[int, Doc("The interval of the box plot, default is 10")] = 10
                    ):
    cp_df = df.copy()

    if n is not None:
        cp_df = cp_df.resample(f'{n}D').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        }).dropna()

    chart_data: list = []
    labels: list = []

    for index, row in cp_df.iterrows():
        chart_data.append([row['Close'], row['Open'], row['Low'], row['High']])
        labels.append(index.strftime('%Y-%m-%d'))

    figure, axes = plt.subplots()
    axes.boxplot(chart_data, vert=True, patch_artist=True)
    axes.set_xticklabels(labels)
    axes.set_title(f'{ticker} Box Plot Chart')
    axes.set_xlabel('Date')
    axes.set_ylabel('Price')
    axes.set_xticks(range(1, len(labels) + 1, k))
    axes.set_xticklabels(labels[::k], rotation=90)

    plt.show()




In [ ]:
draw_box_chart(data, n=None , k=50)

In [ ]:
draw_box_chart(data, n=50, k=10)

In [ ]:
draw_box_chart(data, n=15, k=15)